In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
# --- 1) Database connection & data load ---
db_config = {
    'host':        'localhost',
    'database':    'Liberty',
    'user':        'postgres',
    'password':    'abc',
    'port':        '5432'
}
connection_string = (
    f"postgresql://{db_config['user']}:{db_config['password']}"
    f"@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)
engine = create_engine(connection_string)

# Load full dataset ordered by policy group and date
query = """
SELECT *
FROM insurancedata_with_fb_cc_pc
ORDER BY 
  "Cleaned Chassis Number", 
  "Cleaned Engine Number", 
  "corrected_name", 
  "policy start date", 
  "policy end date";
"""
df = pd.read_sql(text(query), con=engine)

In [2]:
import pandas as pd
import numpy as np
import re

# Dates, dtypes & ordering
df['policy start date'] = pd.to_datetime(df['policy start date'])
df['policy end date']   = pd.to_datetime(df['policy end date'])

# Function to clean names
def clean_name(name):
    return re.sub(r'[^a-zA-Z0-9]', '', str(name)).lower()

# Clean make/model columns
df['rto_location_clean'] = df['rto location'].apply(clean_name)
df['fuel_type_clean'] = df['fuel type'].apply(clean_name)
df['product_name_clean'] = df['product name'].apply(clean_name)
df['vehicle_segment_clean'] = df['vehicle segment'].apply(clean_name)


# Convert high-cardinality object columns to category
cat_cols = [
    'Policy Status', 'Cleaned State2', 'rto_location_clean',
    'model_clean', 'fuel_type_clean', 'make_clean', 'product_name_clean', 'vehicle_segment_clean'
]
for c in cat_cols:
    df[c] = df[c].astype('category')

group_cols = ['Cleaned Chassis Number', 'Cleaned Engine Number', 'corrected_name']
df = df.sort_values(group_cols + ['policy start date', 'policy end date'])

# Renewal flag & active indicator
df['renewal_flag'] = df['Policy Status'].map({'Renewed': 1, 'Not Renewed': 0})
df['is_active']    = df['Policy Status'].eq('Open')

g = df.groupby(group_cols)

# Fast cumulative features
# Historical retention rate
cum_sum   = g['renewal_flag'].cumsum() - df['renewal_flag']
cum_count = g.cumcount()
df['retention_rate_pct'] = np.where(cum_count > 0, cum_sum / cum_count, np.nan)

# Historical average premium
cum_prem  = g['total premium payable'].cumsum() - df['total premium payable']
df['avg_premium_hist'] = np.where(cum_count > 0, cum_prem / cum_count, np.nan)

# Retention streak (vectorised)
df['prev_renew'] = g['renewal_flag'].shift().fillna(0)
df['streak_block'] = (
    (df['prev_renew'] == 0)
    .astype(int)
    .groupby(df[group_cols].apply(tuple, axis=1))
    .cumsum()
)
df['retention_streak'] = (
    df.groupby(group_cols + ['streak_block'])['prev_renew'].cumsum()
)
df.drop(columns=['prev_renew', 'streak_block'], inplace=True)

# Lagged premium & YoY ratio
df['lag_1_premium'] = g['total premium payable'].shift()
df['previous_year_premium_ratio'] = df['total premium payable'] / df['lag_1_premium']

# Days between renewal start dates (regardless of package/overlap)
df['days_between_renewals'] = g['policy start date'].diff().dt.days

# Correct Package/Overlap-Aware Gap Logic
def calc_gaps(d, keys):
    d = d.sort_values(keys + ['policy start date', 'policy end date'])

    # Unique group ID
    d['_gid'] = d.groupby(keys).ngroup()

    # Flag package policies (same start date, second+ occurrence)
    d['_is_pkg'] = d.groupby(['_gid', 'policy start date']).cumcount() > 0

    # Previous unique start (skip same start rows)
    d['_prev_start'] = (
        d.groupby('_gid')['policy start date']
         .transform(lambda x: x.shift().where(x != x.shift()).ffill())
    )

    # Map: (gid, start) → earliest end for that start
    ends = (
        d.groupby(['_gid', 'policy start date'])['policy end date']
         .min()
         .reset_index()
         .rename(columns={
             'policy start date': '_prev_start',
             'policy end date'  : '_prev_min_end'
         })
    )

    # Merge previous end into main table
    d = d.merge(ends, on=['_gid', '_prev_start'], how='left')

    # Compute final gap
    d['days_gap_prev_end_to_curr_start'] = np.where(
        d['_is_pkg'],
        0,
        np.where(
            d['_prev_min_end'].notna(),
            (d['policy start date'] - d['_prev_min_end']).dt.days,
            np.nan
        )
    )

    # Clean-up
    d.drop(columns=['_gid', '_is_pkg', '_prev_start', '_prev_min_end'], inplace=True)
    return d

# APPLY THE GAP FUNCTION HERE
df = calc_gaps(df, group_cols)

# Claim approval rate – safe divide
claim_total = (df['approved'] + df['denied']).replace(0, np.nan)
df['claim_approval_rate'] = df['approved'] / claim_total

# Simple ratios
df['idv_premium_ratio']     = df['vehicle idv'] / df['total premium payable']
df['add_on_adoption']       = df['before gst add-on gwp'] / df['total premium payable'].replace(0, np.nan)
df['od_tp_ratio']           = df['total od premium'] / df['total tp premium'].replace(0, np.nan)

# Lagged technical values
df['lag_1_ncb']        = g['ncb amount'].shift()
df['lag_1_od_premium'] = g['total od premium'].shift()
df['lag_1_tp_premium'] = g['total tp premium'].shift()

# Risk scores (vectorised)
closed = df.loc[~df['Policy Status'].eq('Open'), [
    'renewal_flag', 'Cleaned State2', 'rto_location_clean', 'model_clean', 'vehicle_segment_clean', 'fuel_type_clean', 'product_name_clean',
    'make_clean'
]].copy()

closed['churn_target'] = 1 - closed['renewal_flag']

risk_means = (
    closed.melt(id_vars='churn_target', var_name='risk_dim', value_name='key')
          .groupby(['risk_dim', 'key'])['churn_target'].mean()
          .rename('risk_score')
          .reset_index()
)

# Map each risk dimension back
risk_map = {
    'Cleaned State2': 'state_risk_score',
    'rto_location_clean': 'rto_risk_factor',
    'vehicle_segment_clean': 'segment_risk_score',
    'model_clean': 'model_risk_score',
    'fuel_type_clean': 'fuel_type_risk_factor',
    'product_name_clean': 'product_risk_factor',
    'make_clean': 'manufacturer_risk_rate'
}

for dim, new_col in risk_map.items():
    df = df.merge(
        risk_means.query("risk_dim == @dim")[['key', 'risk_score']]
                  .rename(columns={'key': dim, 'risk_score': new_col}),
        on=dim, how='left'
    )

print(f" Feature engineering complete – rows: {len(df):,}, cols: {df.shape[1]}")

 Feature engineering complete – rows: 2,306,958, cols: 212


In [3]:
# Calculate total revenue (using 'total premium payable') per policy
df['total_revenue'] = df['total premium payable']

# Calculate total revenue per customer 
customer_total_revenue = df.groupby('customerid')['total_revenue'].sum()

# Calculate total number of purchases (policies) per customer
customer_total_purchases = df.groupby('customerid').size()

# Calculate Average Purchase Value (APV) per customer
customer_apv = customer_total_revenue / customer_total_purchases

# Convert 'Overall Churned' to a binary flag for churn rate calculation
# (Creating a customer-level snapshot by dropping duplicate customerids)
df_customer = df.drop_duplicates(subset='customerid', keep='first')
df_customer['Churned_Binary'] = df_customer['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)

# Calculate global churn rate
unique_customers = df_customer['customerid'].nunique()
churned_customers = df_customer[df_customer['Churned_Binary'] == 1]['customerid'].nunique()
churn_rate = churned_customers / unique_customers if unique_customers > 0 else 0

# Calculate the average customer lifespan (ACL) as the inverse of churn rate
average_customer_lifespan = 1 / churn_rate if churn_rate != 0 else np.inf

# Define Average Purchase Frequency (APF) per customer as the number of purchases
customer_apf = customer_total_purchases

# Calculate Customer Lifetime Value (CLV) for each customer
# This simplifies to: total revenue per customer * average_customer_lifespan
customer_clv = customer_total_revenue * average_customer_lifespan

# Create a customer-level metrics DataFrame to merge back with df
customer_metrics_df = pd.DataFrame({
    'customerid': customer_total_revenue.index,
    'Customer_APV': customer_apv.values,
    'Customer_APF': customer_apf.values,  
    'Churn_Rate': churn_rate,             
    'Average_Customer_Lifespan': average_customer_lifespan,  
    'CLV': customer_clv.values
})

# Merge the customer metrics back into the original DataFrame based on customerid
df = df.merge(customer_metrics_df, on='customerid', how='left')

# Verify the merge by displaying the first few rows
print(df.head())

C:\Users\Admin\AppData\Local\Temp\ipykernel_33608\232488038.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customer['Churned_Binary'] = df_customer['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)


                   policy no premium payable without ncb  \
0  '201130050322700128800000                        None   
1  '201130050422700103900000                       18473   
2  '201140020122701119800001                        8102   
3  '201140020123702958201000                        None   
4  '201140020123100158101000                        None   

  add on eligibilty (5th & 6th renewals with ncb)  total premium payable  \
0                                            None                23618.0   
1                                            None                16743.0   
2                                            None                 6167.0   
3                                            None                 6167.0   
4                                            None                 6286.0   

                 product name  last year ncb           biztype  \
0            PrivateCarPolicy            NaN         Roll Over   
1  Private Car Package Policy            0.0      

In [4]:
df.to_sql(
    "policydata_with_fb_cc_pc_newfea_opti_correct",
    engine,
    if_exists="replace",
    index=False,
    chunksize=100000
)

2358